# Random Numbers (on a computer)
With material from http://www-static.etp.physik.uni-muenchen.de/kurs/comp20/ and https://henryiii.github.io/compclass/.

## Example 1: Buffon's needle
* [“Buffon's needle problem”](https://en.wikipedia.org/wiki/Buffon%27s_needle_problem) (Graf G.L.L. von Buffon, 1707 – 1788)
* draw parallel, equidistant lines (distance $d$), throw $N$ needles (length $l\leq d$)
* probability for needle intersecting with line: $p_\text{hit}(\varphi) = l_\text{eff}/d = l|\cos \varphi|/d$
* integrate, assuming flat distribution for angle $\varphi$: 
$$p = \int p_\text{hit}(\varphi) d \varphi 
    = \int_0^{2\pi} \frac {l \left|\cos {\varphi}\right|}{d} \frac {\mathrm d \varphi}{2\pi}
    = \frac {l}{2\pi d} 2 \int_{-\pi/2}^{\pi/2} \cos\varphi \, \mathrm d \varphi
    = \frac {2l}{\pi d}$$
* [von Mises](https://de.wikipedia.org/wiki/Richard_von_Mises): $N_\text{hits}/N \to p$ for $N\to\infty$ 
* use this to derive an estimate of $\pi$ from counting $N_\text{hits}$ as a simple Monte Carlo method:
$$\tfrac{2\cdot N \cdot l}{N_\text{hits} \cdot d} \rightarrow \pi$$

[![](../figures/Buffon_Streicholz_1.jpg)](https://en.wikipedia.org/wiki/File:Streicholz-Pi.jpg)

<div class="alert alert-block alert-success">
    <b>Question</b>: What do you get for $\pi$ here?
</div>

## Example 2: area of a circle
* also here, we determine $\pi$, but this time from the numerical integration of the area of a circle
* approximate $\pi$ as ratio of hits $t$ and tries $n$:
  * $\lim_{n\to\infty} t/n = \pi / 4$ 
  * Why does this hold?
  
![quarter of a unit circle in a square](../figures/quarter_circle.svg)

How to define a circle:
* circle = shape consisting of all points in a plane at a given distance from the center
* unit circle, using Euclidean distance in 2-D: $x^2 + y^2 = 1$

Implementation:

In [ ]:
import numpy as np
xy    = np.random.rand(2, 10000)
xy2   = np.sum(xy ** 2, axis=0)
valid = xy2 <= 1 # mask to select points within the unit circle
good  = xy[:,  valid]
bad   = xy[:, ~valid]

<div class="alert alert-block alert-info">
    (Think about why we need the <code>axis=0</code> here. What happens without?)
</div>

In [ ]:
import matplotlib.pyplot as plt
plt.plot(*good, ".")
plt.plot(*bad, ".");
#plt.axis("equal");

In [ ]:
np.mean(valid) * 4

Can easily be generalized to higher dimensions:
* e.g. for a sphere: $x^2+y^2+z^2 \leq 1$
* N dimensions: $\sum_{i=1}^N x_i^2 \leq 1$

Volume of a sphere:

In [ ]:
xyz   = np.random.rand(3, 10000)
valid = np.sum(xyz ** 2, axis=0) < 1
good  = xyz[:, valid]
bad   = xyz[:, ~valid]
print("MC:      ", np.mean(valid)*8) # explain: why MC?
print("Analytic:", 4 / 3 * np.pi)

Conclusion: random numbers are an important ingredient in many applications in science, in particular whenever Monte-Carlo (MC) methods can help to solve otherwise untractable problems (e.g. integrals where no analytical solution is known or over spaces with very high dimensionality). 

[MC algorithm](https://en.wikipedia.org/wiki/Monte_Carlo_algorithm) = "randomized algorithm whose output may be incorrect with a certain (typically small) probability"
* less abstract: numerical method to compute probabilities and deduced quantities using random numbers
* mainly used for optimization, numerical integration, or generating values from a probability distribution
  * used in statistical physics, <!-- canonical ensembles --> 
    biophysics, <!-- folding of large molecules like proteins -->
    particle physics, <!-- GEANT4, simulation of particle collisions and particles interacting with matter -->
    economics, <!-- risk assessment -->
    ...
  * useful for systems with many degrees of freedom (fluids, disordered materials, particle physics, ...)
  * advantages: more efficient, easier to understand and implement
  * disadvantage: may be wrong
  * repetition reduces probability of false outcome

Basic principle:
* do a large number of repeated randomized experiments
* rely on [(weak law) of large numbers](https://en.wikipedia.org/wiki/Law_of_large_numbers#Weak_law)
  * $\lim\limits_{n\to\infty} P\left( \left| \overline X_n \right| \geq \epsilon\right) = 0$ 
  * for $\overline X_n = \frac1n \sum\limits_{i=1}^n {\left(X_i - E(X_i)\right)}$, where
    $X_1, X_2, \dots$ = infinite sequence of independent and identically distributed (i.i.d.) random variables (i.e., $E(X_i) = \mu \forall i$)
<!--   * e.g. $X \sim \text{Ber}(p)$, $E(X_i) = p$ -->

## Generating random numbers
We have seen that random numbers are useful / necessary to do (Monte Carlo) integrations on a computer. What is behind the `np.random` module, i.e. how can we construct random numbers using a deterministic system like a computer?

* on a deterministic computer, we can only generate *pseudo-random numbers*
  * special hardware for true random numbers (important for secure cryptography: attacker must not be able to infer random numbers) 
  * pseudo-random numbers sufficient for our purposes (e.g. MC integration)
  * get true random numbers for free [here](https://www.random.org/)
* simplest distribution is a flat distribution
  * start by implementing a generator for random numbers with flat distribution
  * will then see that we can generate random numbers following arbitrary distributions (Gaussian, Poissonian, exponential, ...) from this
* to be suitable for scientific applications / MC methods, we want pseudo-random numbers to 
  * random (sufficiently independent and statistically balanced; passing tests for randomness)
  * have a long period
  * be fast to compute, easy on memory
  * be reproducible (for debugging)

### Simple random-number generator: [linear congruential generator](https://en.wikipedia.org/wiki/Linear_congruential_generator)
* recursive definition: $I_j=(a\cdot I_{j-1}+c) \mod m$ 
  * 3 integer constants: multiplier $a$, increment $c$, modulus $m$ 
  * plus: start (or seed) value $I_0$ (*random seed*)
* generates sequence $I_1,I_2,...$ with $0 \leq I_j \leq m-1$
  * $I_j$ is periodic sequence with (maximum) period $m$
  * $u_j = I_j/m \in [0, 1)$

Let's implement a linear congruential generator as a true `generator` in Python:

In [ ]:
def lin_cong_iter(c, a, m, I_0):
    """implements a linear congruential generator"""
    I_j = I_0

    while True:
        yield I_j # <- Python generator
        I_j = (a * I_j + c) % m
        if I_j == I_0:
            # arrived at seed value again
            yield I_j
            break

(`yield` is like `return` in that the function returns a value when the interpreter encounters `yield`/`return`, but unlike `return`, the interpreter will store the state of the function and continue after `yield` when you call the function again. A function with `yield` statement returns a *lazy iterator*, i.e. you can loop over them (like a list) but they do not carry out the computation until asked for the result.)

In [ ]:
# returns a generator object
lin_cong_iter(0, 3, 7, 1)

In [ ]:
list(lin_cong_iter(0, 3, 7, 1))

Do these look random? Can you predict the next number from the previous numbers without knowing the implementation?

Map to range $[0,1)$:

In [ ]:
list(i / 7 for i in lin_cong_iter(0, 3, 7, 1)) # special case c=0: "multiplicative congruential generator"

### Testing randomness
* correlations will spoil MC computations, results will be biased (i.e. wrong)
* define suite of tests (e.g. [TestU01](https://en.wikipedia.org/wiki/TestU01)) that random-number generators have to pass, e.g.
  * test flatness of distribution ($\chi^2$ tests for sub-intervals of $[0,1)$)
  * correlation tests ([spectral tests](https://en.wikipedia.org/wiki/Spectral_test))
  
Let's do a spectral test of our LCG:

In [ ]:
# generate some random numbers
lcg_numbers = list(lin_cong_iter(a=57, c=1, m=256, I_0=10))[:-1]
print(len(lcg_numbers))

In [ ]:
# numpy's random numbers
real_rand = np.random.randint(low = 0, high = 256, size = len(lcg_numbers))

In [ ]:
# plot sequence
fig, axs = plt.subplots(1, 2, figsize=(16, 4))
axs[0].plot(range(len(lcg_numbers)), lcg_numbers, "x-", label = "lcg_numbers")
axs[0].legend()
axs[1].plot(range(len(lcg_numbers)), real_rand, "x-", label = "numpy")
axs[1].legend()
plt.show()

No visible difference.

In [ ]:
# flatness
plt.hist(lcg_numbers, bins = 20, range = (0, 255), alpha = 0.5, label = "lcg_numbers")
plt.hist(real_rand, bins = 20, range = (0, 255), alpha = 0.5, label = "numpy")
plt.legend()
plt.show()

## 🤔

<div class="alert alert-block alert-info">
    (Think about the expected size of the fluctuations for 256 total entries, taking into account that we have 20 bins.)
</div>

In [ ]:
# numpy's random numbers
np.random.seed(105)
real_rand = np.random.randint(low = 0, high = 256, size = len(lcg_numbers))
# spectral test in 2-D
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
axs[0].plot(lcg_numbers[::2], lcg_numbers[1::2], "+", label = "lcg_numbers")
axs[0].legend()
axs[1].plot(real_rand[::2], real_rand[1::2], "+", label = "numpy")
axs[1].legend()
plt.show()

## 😲

## Random numbers with arbitrary distributions
Now that we have a generator for i.i.d. random numbers, let's get back to the point of transforming these into random numbers for arbitrary probability distributions. We will look at:
* inverse transform sampling (analytic, thus efficient)
* rejection sampling (useful if inversion of PDF not possible)
* special cases (e.g. Gaussian random numbers)

### Inverse transform sampling
The cumulative distribution function (CDF) $F$ of a random variable $X$ is defined as
$F_{X}(x) \dot= \operatorname{Prob}(X \leq x)$.

The CDF for a probability density function (PDF) $f$ can be computed as 
$F(x): x \to [0,1], F(x) \dot{=} \int_{-\infty}^x f(t) {\mathrm dt}$.
	
For the inverse transform sampling, we need to compute the inverse $F^{-1}$ of the CDF $F$ of the PDF $f$ that we want to sample from.

Some PDFs for normal distributions:
![](../figures/Normal_Distribution_PDF.png) 

Corresponding CDFs:
![](../figures/Normal-distribution-cumulative-density-function-many.png)

[Inverse transform sampling](https://en.wikipedia.org/wiki/Inverse_transform_sampling): 
* $F^{-1}: [0,1] \to \operatorname{domain}(f)$ gives random numbers $X$ distributed according to $f$ when using i.i.d. random numbers $U$ as input, $X = F^{-1}(U)$.
* [proof](https://en.wikipedia.org/wiki/Inverse_transform_sampling#Proof_of_correctness)

### Example: exponentially distributed random numbers
* PDF: $f(x) = \lambda \cdot \exp(-\lambda \cdot x)$ for $x\geq0$
* CDF: $F(x) = \int_{t_\text{min} = 0}^{x}\lambda \cdot \exp(-\lambda\cdot t) \mathrm dt = [-\exp(-\lambda \cdot t)]_0^x =  1 - \exp(-\lambda \cdot x)$ 
* inverse: $x(u) = F^{-1}(x) = -\ln(1-u)/\lambda = -\ln(u)/\lambda$

In [ ]:
# generate
random_u = np.random.random(300000)
_lambda  = 2
random_x = -np.log(random_u)/_lambda

# plot
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
axs[0].hist(random_u, bins = 30)
axs[1].hist(random_x, bins = 30)
axs[1].set_yscale('log')
plt.show()

In [ ]:
# detour: how to invert with sympy
import sympy
t, x, l, u = sympy.symbols("t x lambda u")
f = l*sympy.exp(-l*t)
f

In [ ]:
F = sympy.integrate(f, (t, 0, x))
F

In [ ]:
from sympy import solve, simplify
solve(F-u, x)[0]

<div class="alert alert-block alert-info">
    (Is this the same as our result? Which assumption have we made above that <code>sympy</code> does not know about?)
</div>

### Rejection sampling (Example: Gaussian distributed random numbers)
For a Gaussian distribution $f(x) = \exp(-x^2/2) / \sqrt{2\pi}$, there is no analytic inverse of the CDF, so we cannot use inverse transform sampling. Instead, we will introduce [rejection sampling](https://en.wikipedia.org/wiki/Rejection_sampling). The principle is the same as what we used to compute the area of a circle above. In its simplest form:
* define the range to draw samples from, $[x_\text{min}, x_\text{max}]$
* determine the maximum value of the PDF, $f_\text{max}$, over this range
* draw two i.i.d. random numbers $u_1$, $u_2$, map $u_1$ to $u_1' \in [x_\text{min}, x_\text{max}]$ and $u_2$ to $u_2' \in [0, f_\text{max}]$
* if $u_2' < f(u_1')$, accept $u_1'$ as a sample from $f(x)$, otherwise repeat

In [ ]:
def f(x):
    # note: ufunc
    return np.exp(-x**2/2) / np.sqrt(2*np.pi)

# ranges
xmin  = -3
xmax  = +3
fmax  = 0.5
# uniforms
xs    = xmin + np.random.random(10000) * (xmax-xmin) # u1'
ys    = np.random.random(10000) * fmax # u2'
# accept or reject
valid = ys < f(xs)

In [ ]:
# plot
plt.plot(xs[ valid], ys[ valid], ".")
plt.plot(xs[~valid], ys[~valid], ".");

This is quite inefficient, in particular if the fraction of our sampling box covered by the area under $f(x)$ is small:

In [ ]:
print("Acceptance rate:", sum(valid) / len(valid))

Ideas to improve this are *importance sampling* (sample subranges with large $f(x)$ with higher frequency) or to approximate the function with a known (strictly smaller) analytic function and draw samples from this (and the remainder if large).

For special cases, there may be more efficient alternatives. One example for the Gaussian distribution is the [Box-Muller method](https://en.wikipedia.org/wiki/Box%E2%80%93Muller_transform), which can produce 2 random Gaussian-distributed numbers from 2 i.i.d. input random numbers:

In [ ]:
def BoxMuller(u1, u2):
    """Box–Muller transform (basic form)"""
    # two random numbers in, two out
    f1 = np.sqrt(-2*np.log(u1))
    f2 = 2*np.pi*u2
    gauss1 = f1 * np.cos(f2)
    gauss2 = f1 * np.sin(f2)
    return gauss1, gauss2

In [ ]:
u1 = np.random.random(10000)
u2 = np.random.random(10000)
g = BoxMuller(u1, u2)
#plt.hist(u2, bins = 20);
plt.hist(np.concatenate(g), bins = 20);

<div class="alert alert-block alert-success">
    Task: do a fit of this distribution with a Gaussian (we'll discuss fitting in the next notebook).)
</div>